# Konfiguracja

In [1]:
# Narzędzie jest oparte o kilka zewnętrznych bibliotek, które potrzebujemy najpierw zaimportować.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

# Parametr określający jak blisko (w odsetku całego widocznego zakresu) punktu początkowego 
# wielokąta musimy kliknąć, aby go zamknąć.
TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

# Klasa ta trzyma obecny stan wykresu oraz posiada metody, które mają zostać wykonane
# po naciśnięciu przycisków.
class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax
        
    # Metoda ta obsługuje logikę przejścia do następnej sceny.
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    # Metoda ta obsługuje logikę powrotu do poprzedniej sceny.
    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)
        
    # Metoda ta aktywuje funkcję rysowania punktów wyłączając równocześnie rysowanie 
    # odcinków i wielokątów.
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))
            
    # Metoda ta aktywuje funkcję rysowania odcinków wyłączając równocześnie
    # rysowanie punktów i wielokątów.     
    def add_line(self, event):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    # Metoda ta aktywuje funkcję rysowania wielokątów wyłączając równocześnie
    # rysowanie punktów i odcinków.
    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []
        
    # Metoda odpowiedzialna za właściwą logikę rysowania nowych elementów. W
    # zależności od włączonego trybu dodaje nowe punkty, początek, koniec odcinka
    # lub poszczególne wierzchołki wielokąta. Istnieje ciekawa logika sprawdzania
    # czy dany punkt jest domykający dla danego wielokąta. Polega ona na tym, że
    # sprawdzamy czy odległość nowego punktu od początkowego jest większa od
    # średniej długości zakresu pomnożonej razy parametr TOLERANCE.   
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:    
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)
    
    # Metoda odpowiedzialna za narysowanie całego wykresu. Warto zauważyć,
    # że zaczyna się ona od wyczyszczenia jego wcześniejszego stanu. Istnieje w
    # niej nietrywialna logika zarządzania zakresem wykresu, tak żeby, w zależności
    # od ustawionego parametru autoscaling, uniknąć sytuacji, kiedy dodawanie
    # nowych punktów przy brzegu obecnie widzianego zakresu powoduje niekorzystne
    # przeskalowanie.
    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()


In [2]:
# Klasa Scene odpowiada za przechowywanie elementów, które mają być
# wyświetlane równocześnie. Konkretnie jest to lista PointsCollection i
# LinesCollection.
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

# Klasa PointsCollection gromadzi w sobie punkty jednego typu, a więc takie,
# które zostaną narysowane w takim samym kolorze i stylu. W konstruktorze
# przyjmuje listę punktów rozumianych jako pary współrzędnych (x, y). Parametr
# kwargs jest przekazywany do wywołania funkcji z biblioteki MatPlotLib przez
# co użytkownik może podawać wszystkie parametry tam zaproponowane.        
class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points = self.points + points

# Klasa LinesCollection podobnie jak jej punktowy odpowiednik gromadzi
# odcinki tego samego typu. Tworząc ją należy podać listę linii, gdzie każda
# z nich jest dwuelementową listą punktów – par (x, y). Parametr kwargs jest
# przekazywany do wywołania funkcji z biblioteki MatPlotLib przez co użytkownik
# może podawać wszystkie parametry tam zaproponowane.
class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

# Klasa Plot jest najważniejszą klasą w całym programie, ponieważ agreguje
# wszystkie przygotowane sceny, odpowiada za stworzenie wykresu i przechowuje
# referencje na przyciski, dzięki czemu nie będą one skasowane podczas tzw.
# garbage collectingu.
class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
    
    # Ta metoda ma szczególne znaczenie, ponieważ konfiguruje przyciski i
    # wykonuje tym samym dość skomplikowaną logikę. Zauważmy, że konfigurując każdy
    # przycisk podajemy referencję na metodę obiektu _Button_callback, która
    # zostanie wykonana w momencie naciśnięcia.
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    # Metoda toJson() odpowiada za zapisanie stanu obiektu do ciągu znaków w
    # formacie JSON.
    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])    
    
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None
        
    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None
    
    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None
    
    # Główna metoda inicjalizująca wyświetlanie wykresu.
    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()
        

# Utility

In [3]:
from copy import deepcopy
from queue import PriorityQueue

In [4]:
def generate_points(amount, left , right):
    array=[None]*amount
    for i in range(amount):
        array[i]=[np.random.uniform(left, right),np.random.uniform(left, right)]
    return array

In [5]:
class Point:
   x = 0.0
   y = 0.0
   
   def __init__(self, x, y):
       self.x = x
       self.y = y

class Event:
    x = 0.0
    p = None
    a = None
    valid = True
    
    def __init__(self, x, p, a):
        self.x = x
        self.p = p
        self.a = a
        self.valid = True

class Arc:
    p = None
    pprev = None
    pnext = None
    e = None
    s0 = None
    s1 = None
    
    def __init__(self, p, a=None, b=None):
        self.p = p
        self.pprev = a
        self.pnext = b 
        self.e = None
        self.s0 = None
        self.s1 = None

class Segment:
    start = None
    end = None
    done = False
    
    def __init__(self, p):
        self.start = p
        self.end = None
        self.done = False

    def finish(self, p):
        if self.done: return
        self.end = p
        self.done = True        

def peek(que: PriorityQueue):
    priority, item = que.get()
    que.put((priority, item))
    return item

In [6]:
e = 10**(-12)

In [7]:
def det_3by3(a, b, c):
    '''calculate where point c is located in accordance to vector a-b
    if det > 0, point is on the left
    if < 0 then point on the right
    if 0 then colineal'''


    return a[0]*b[1]+a[1]*c[0]+b[0]*c[1]-c[0]*b[1]-a[1]*b[0]-a[0]*c[1]

In [8]:
def orient(a, b, c):
    '''Orient returns 1 if point c is on the left of the lina a->b, 
    -1 if c is on the right 
    and 0 if the're collineal '''
    d = det_3by3(a, b, c)
    if d > e:
        return 1
    elif d < (-1)*e:
        return -1
    else:
        return 0

In [9]:
def orient_points(a: Point, b: Point, c: Point):
    '''Same as regular orient but works with Point class. 
    Orient returns 1 if point c is on the left of the lina a->b, 
    -1 if c is on the right 
    and 0 if the're collineal '''


    d = det_3by3((a.x, a.y), (b.x, b.y), (c.x, c.y))
    if d > e:
        return 1
    elif d < (-1)*e:
        return -1
    else:
        return 0

In [10]:
def bounding_box(points):
    '''returns a tuple conataining a pair of vectors, lower left edge of point set an upper right'''
    min_x = float('inf')
    min_y = float('inf')
    max_x = -float('inf')
    max_y = -float('inf')

    n = len(points)

    for i in range(n):
        if points[i][0] < min_x:
            min_x = points[i][0]

        if points[i][1] < min_y:
            min_y = points[i][1]

        if points[i][0] > max_x:
            max_x = points[i][0]

        if points[i][1] > max_y:
            max_y = points[i][1]

    width = max_x - min_x
    height = max_y - min_y

    result = ((min_x - 0.5*width, min_y - 0.5*height), (max_x + 0.5*width, max_y + 0.5*height))

    return result

In [11]:
def distance(a, b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

In [12]:
def find_circle_center(a, b, c):
    Ax, Ay = a
    Bx, By = b
    Cx, Cy = c
    D = 2*(Ax*(By-Cy)+Bx*(Cy-Ay)+Cx*(Ay-By))
    ox = 1/D * ((Ax**2+Ay**2)*(By-Cy)+(Bx**2+By**2)
                * (Cy-Ay)+(Cx**2+Cy**2)*(Ay-By))
    oy = 1/D * ((Ax**2+Ay**2)*(Cx-Bx)+(Bx**2+By**2)
                * (Ax-Cx)+(Cx**2+Cy**2)*(Bx-Ax))
    return (ox, oy)

In [13]:
def calculate_radius(A, B, C):
    a, b, c = distance(A, B), distance(B, C), distance(C, A)
    s = (a+b+c)/2
    return a * b * c/(4 * np.sqrt(s * (s-a) * (s-b) * (s-c)))

In [14]:
def is_in_circle(point, triangle, points):
    '''check whether the point is inside the circumcircle of a traingle'''
    p = points[point]
    a, b, c = points[triangle[0]], points[triangle[1]], points[triangle[2]]

    center = find_circle_center(a, b, c)
    d = distance(p, center)
    r = calculate_radius(a, b, c)

    return d < r or abs(d- r) < e

# Delaunay Triangulation

In [67]:
def triangulate(points):

    n = len(points)
    triangles = set()

    low_left, up_right = bounding_box(points)

    edge_dict = {}

    # bounding box 
    # d ------ c
    # |        |
    # |        |
    # a ------ b
    # add the bounding box to the points and remember their indexes
    points.append(low_left)
    a = n
    points.append((up_right[0], low_left[1]))
    b = n + 1
    points.append(up_right)
    c = n + 2
    points.append((low_left[0], up_right[1]))
    d = n + 3
    

    super_t_1 = (a, b, d)
    super_t_2 = (b, c, d)

    triangles.add(super_t_1)
    triangles.add(super_t_2)

    edge_dict = {(a, b): {super_t_1},
                (b, d): {super_t_1, super_t_2},
                (a, d): {super_t_1},
                (b, c): {super_t_2},
                (c, d): {super_t_2}}

    for point in range(n):
        badTriangles = []
        badEdges = set()

        for triangle in triangles:
            if is_in_circle(point, triangle, points):
                badTriangles.append(triangle)

                edges = [(triangle[0], triangle[1]), (triangle[1], triangle[0]), (triangle[1], triangle[2]), (triangle[2], triangle[1]), (triangle[0], triangle[2]), (triangle[2], triangle[0])]
                for i in range(0, 3):
                    edgeAndRev = {edges[i*2], edges[i*2+1]}
                    if len(badEdges.intersection(edgeAndRev)) == 0:
                        badEdges.add(edges[i*2])
                    else:
                        badEdges = badEdges.difference(edgeAndRev)

        for triangle in badTriangles:
            triangles.remove(triangle)
            t_edges = (triangle[0], triangle[1]), (triangle[1], triangle[2]), (triangle[2], triangle[0])
            for edge in t_edges:
                if (s := edge_dict.get(edge)) != None:
                    s.remove(triangle)
                    if len(edge_dict[edge]) == 0:
                        del edge_dict[edge]
                else:
                    s = edge_dict[edge[::-1]]
                    s.remove(triangle)
                    if len(edge_dict[edge[::-1]]) == 0:
                        del edge_dict[edge[::-1]]

        for edge in badEdges:
            triangle = (edge[0], edge[1], point)
            triangles.add(triangle)
            t_edges = (triangle[0], triangle[1]), (triangle[1], triangle[2]), (triangle[2], triangle[0])
            for edge in t_edges:
                if edge_dict.get(edge) != None:
                    edge_dict[edge].add(triangle)
                elif edge_dict.get(edge[::-1]) != None:
                    edge_dict[edge[::-1]].add(triangle)
                else:
                    edge_dict[edge] = {triangle}

    badTriangles = set()
    for triangle in triangles:
        for point in super_t_1:
            if point in triangle:
                badTriangles.add(triangle)
                break

        for point in super_t_2:
            if point in triangle:
                badTriangles.add(triangle)
                break

    for triangle in badTriangles:
        triangles.remove(triangle)


    v_diagram = []

    for edge in edge_dict:
        t_s = edge_dict[edge].intersection(triangles)
        if len(t_s) == 2:
            t_1, t_2 = t_s.pop(), t_s.pop()
        elif len(t_s) == 1:
            t_1, t_2 = edge_dict[edge].pop(), edge_dict[edge].pop()
        else:
            t_1, t_2 = None, None

        if t_1 != None and t_2 != None:
            a_1, b_1, c_1 = points[t_1[0]], points[t_1[1]], points[t_1[2]]
            a_2, b_2, c_2 = points[t_2[0]], points[t_2[1]], points[t_2[2]]
            v_diagram.append((find_circle_center(a_1, b_1, c_1), find_circle_center(a_2, b_2, c_2)))

    # triangles = change_triangles_to_points(triangles, points)

    return triangles, v_diagram

def change_triangles_to_points(triangles, points):
    '''change triagnles from indexes in point list to actual float values of points'''
    result = []
    for t in triangles:
        triangle = points[t[0]], points[t[1]], points[t[2]]
        result.append(triangle)

    return result

def triang_to_lines(triangles):

    lines = []
    for triangle in triangles:
        lines.append((triangle[0], triangle[1]))
        lines.append((triangle[1], triangle[2]))
        lines.append((triangle[2], triangle[0]))

    return lines    


# Fortune's Algorithm

In [65]:
class Voronoi:
    def __init__(self, points):
        self.points_copy = points
        self.output = [] # list of line segment
        self.arc = None  # binary tree for parabola arcs

        self.points = PriorityQueue() # site events
        self.event = PriorityQueue() # circle events

        self.scenes = [] # empty scene list

        # bounding box
        box = self.bounding_box(self.points_copy)
        self.x0 = box[0][0]
        self.x1 = box[1][0]
        self.y0 = box[0][1]
        self.y1 = box[1][1]

        # insert points to site event
        for p in points:
            point = Point(p[0], p[1])
            self.points.put((point.x, point))

    def create_diagram(self):
        while not self.points.empty():
            p = peek(self.points)
            c = float('inf')
            if not self.event.empty():
                c = peek(self.event)
            if not self.event.empty() and (c.x <= p.x):
                self.process_event() # handle circle event
            else:
                self.process_point() # handle site event

        # after all points, process remaining circle events
        while not self.event.empty():
            self.process_event()

        # complete any unfinished segments
        self.finish_edges()


    def process_point(self):
        _, p = self.points.get()

        self.arc_insert(p)

    def process_event(self):
        _, e = self.event.get()

        if e.valid:
            # start new edge
            s = Segment(e.p)
            self.output.append(s)

            # remove associated arc (parabola)
            a = e.a
            if a.pprev is not None:
                a.pprev.pnext = a.pnext
                a.pprev.s1 = s
            if a.pnext is not None:
                a.pnext.pprev = a.pprev
                a.pnext.s0 = s

            # finish the edges before and after a
            if a.s0 is not None: 
                a.s0.finish(e.p)
            if a.s1 is not None: 
                a.s1.finish(e.p)

            # recheck circle events on either side of p
            if a.pprev is not None: 
                self.check_circle_event(a.pprev, e.x)
            if a.pnext is not None: 
                self.check_circle_event(a.pnext, e.x)

    def arc_insert(self, p):
        if self.arc is None:
            self.arc = Arc(p)
        else:
            # find the current arcs at p.y
            i = self.arc
            while i is not None:
                flag, z = self.intersect(p, i)
                if flag:
                    # new parabola intersects arc i
                    flag, zz = self.intersect(p, i.pnext)
                    if (i.pnext is not None) and (not flag):
                        i.pnext.pprev = Arc(i.p, i, i.pnext)
                        i.pnext = i.pnext.pprev
                    else:
                        i.pnext = Arc(i.p, i)
                    i.pnext.s1 = i.s1

                    # add p between i and i.pnext
                    i.pnext.pprev = Arc(p, i, i.pnext)
                    i.pnext = i.pnext.pprev

                    i = i.pnext # now i points to the new arc

                    # add new half-edges connected to i's endpoints
                    seg = Segment(z)
                    self.output.append(seg)
                    i.pprev.s1 = i.s0 = seg

                    seg = Segment(z)
                    self.output.append(seg)
                    i.pnext.s0 = i.s1 = seg

                    # check for new circle events around the new arc
                    self.check_circle_event(i, p.x)
                    self.check_circle_event(i.pprev, p.x)
                    self.check_circle_event(i.pnext, p.x)

                    return
                        
                i = i.pnext

            # if p never intersects an arc, append it to the list
            i = self.arc
            while i.pnext is not None:
                i = i.pnext
            i.pnext = Arc(p, i)
            
            # insert new segment between p and i
            x = self.x0
            y = (i.pnext.p.y + i.p.y) / 2.0
            start = Point(x, y)

            seg = Segment(start)
            i.s1 = i.pnext.s0 = seg
            self.output.append(seg)

    def check_circle_event(self, i, x0):
        # look for a new circle event for arc i
        if (i.e is not None) and (i.e.x  != self.x0):
            i.e.valid = False
        i.e = None

        if (i.pprev is None) or (i.pnext is None): return

        flag, x, o = self.circle(i.pprev.p, i.p, i.pnext.p)
        if flag and (x > self.x0):
            i.e = Event(x, o, i)
            self.event.put((i.e.x, i.e))

    def circle(self, a, b, c):
        # check if bc is a "right turn" from ab
        if orient_points(a, b, c) > 0: return False, None, None

        if orient_points(a, b, c) == 0: return False, None, None # Points are co-linear

        A = b.x - a.x
        B = b.y - a.y
        C = c.x - a.x
        D = c.y - a.y
        E = A*(a.x + b.x) + B*(a.y + b.y)
        F = C*(a.x + c.x) + D*(a.y + c.y)
        G = 2*(A*(c.y - b.y) - B*(c.x - b.x))

        # calculate the center of the circle
        ox = 1.0 * (D*E - B*F) / G
        oy = 1.0 * (A*F - C*E) / G

        # o.x plus radius equals max x coord
        x = ox + np.sqrt((a.x-ox)**2 + (a.y-oy)**2)
        o = Point(ox, oy)
           
        return True, x, o
        
    def intersect(self, p, i):
        # check whether a new parabola at point p intersect with arc i
        if (i is None): return False, None
        if (i.p.x == p.x): return False, None

        a = 0.0
        b = 0.0

        if i.pprev is not None:
            a = (self.intersection(i.pprev.p, i.p, p.x)).y
        if i.pnext is not None:
            b = (self.intersection(i.p, i.pnext.p, p.x)).y

        if (((i.pprev is None) or (a <= p.y)) and ((i.pnext is None) or (p.y <= b))):
            py = p.y
            px = 1.0 * ((i.p.x)**2 + (i.p.y-py)**2 - p.x**2) / (2*i.p.x - 2*p.x)
            res = Point(px, py)
            return True, res
        return False, None

    def intersection(self, p0, p1, l):
        # get the intersection of two parabolas
        p = p0
        if (p0.x == p1.x):
            py = (p0.y + p1.y) / 2.0
        elif (p1.x == l):
            py = p1.y
        elif (p0.x == l):
            py = p0.y
            p = p1
        else:
            # use quadratic formula
            z0 = 2.0 * (p0.x - l)
            z1 = 2.0 * (p1.x - l)

            a = 1.0/z0 - 1.0/z1
            b = -2.0 * (p0.y/z0 - p1.y/z1)
            c = 1.0 * (p0.y**2 + p0.x**2 - l**2) / z0 - 1.0 * (p1.y**2 + p1.x**2 - l**2) / z1

            py = 1.0 * (-b-np.sqrt(b*b - 4*a*c)) / (2*a)
            
        px = 1.0 * (p.x**2 + (p.y-py)**2 - l**2) / (2*p.x-2*l)
        res = Point(px, py)
        return res

    def finish_edges(self):
        l = self.x1 + (self.x1 - self.x0) + (self.y1 - self.y0)
        i = self.arc
        while i.pnext is not None:
            if i.s1 is not None:
                p = self.intersection(i.p, i.pnext.p, 2*l)
                p = self.is_p_in_bound(p, i.s1)
                i.s1.finish(p)
                self.add_scene(Point(self.x1, self.y1))
            i = i.pnext

    def is_p_in_bound(self, p: Point, line: Segment):
        if line.start.x > self.x1 or line.start.x < self.x0:
            if line.start.y > self.y1 or line.start.y < self.y0:
                return line.start

        point_b = p.x, p.y
        point_a = line.start.x, line.start.y
        new_x = p.x
        new_y = p.y

        a_1 = (point_b[1] - point_a[1])/(point_b[0] - point_a[0])
        b_1 = point_a[1] - a_1*point_a[0]

        if self.x0 <= p.x <= self.x1:
            new_x = p.x
        else:
            if p.x < self.x0:
                new_x = self.x0
            if p.x > self.x1:
                new_x = self.x1

        if self.y0 <= p.y <= self.y1:
            new_y = a_1*new_x + b_1
        else:
            if p.y < self.y0:
                new_y = self.y0
                new_x = (new_y - b_1) / a_1
            if p.y > self.y1:
                new_y = self.y1
                new_x = (new_y - b_1) / a_1


        return Point(new_x, new_y)


            
    def add_scene(self, event):
        p, l = self.get_current_scene()
        self.scenes.append(Scene(points=[PointsCollection(self.points_copy), PointsCollection(p, color = "red")],
        lines=[LinesCollection(l, color = "green"), LinesCollection([[(event.x, self.y0), (event.x, self.y1)]], color = "red")]))

    def get_current_scene(self):
        points = []
        lines = []
        for item in self.output:
            if item.done:
                line = ((item.start.x, item.start.y), (item.end.x, item.end.y))
                lines.append(line)
            else:
                point = (item.start.x, item.start.y)
                points.append(point)

        return points, lines
        

    def point_list(self):
        res = []
        for o in self.output:
            p_0 = o.start
            res.append((p_0.x, p_0.y))

        return res
    def segment_to_list(self):
        result = []
        for o in self.output:
            p0 = o.start
            p1 = o.end
            result.append(((p0.x, p0.y), (p1.x, p1.y)))

    def print_output(self):
        it = 0
        for o in self.output:
            it = it + 1
            p0 = o.start
            p1 = o.end
            print (p0.x, p0.y, p1.x, p1.y)

    def get_output(self):
        res = []
        for o in self.output:
            p0 = o.start
            p1 = o.end
            res.append(((p0.x, p0.y), (p1.x, p1.y)))
            
        ret = LinesCollection(lines=res)    
        return ret

    def bounding_box(self, points):
        '''returns a tuple conataining a pair of vectors, lower left edge of point set an upper right with added 20% margins'''
        min_x = float('inf')
        min_y = float('inf')
        max_x = -float('inf')
        max_y = -float('inf')

        n = len(points)

        for i in range(n):
            if points[i][0] < min_x:
                min_x = points[i][0]

            if points[i][1] < min_y:
                min_y = points[i][1]

            if points[i][0] > max_x:
                max_x = points[i][0]

            if points[i][1] > max_y:
                max_y = points[i][1]

        width = max_x - min_x
        height = max_y - min_y

        result = (min_x - 0.5*width, min_y - 0.5*height), (max_x + 0.5*width, max_y + 0.5*height)

        return result

# Time Tests

In [20]:
from time import time

In [71]:
a = generate_points(10, -10, 10)
b = generate_points(100, -10, 10)
c = generate_points(1000, -10, 10)
d = generate_points(2000, -10, 10)

In [62]:
def count_time(point_set):
    v_points = deepcopy(point_set)
    start_f = time()
    v = Voronoi(v_points)
    v.create_diagram()
    end_f = time()

    d_points = deepcopy(point_set)
    start_d = time()
    triangulate(d_points)
    end_d = time()

    fortune_t = (end_f - start_f)
    delaunay_t = end_d - start_d

    print("Fortune's algorith time is: ", round(fortune_t, 3))
    print("Delaunay's time is: ", round(delaunay_t, 3))

    print("-"*10)




In [72]:

count_time(a)
count_time(b)
count_time(c)
count_time(d)


Fortune's algorith time is:  0.001
Delaunay's time is:  0.001
----------
Fortune's algorith time is:  0.014
Delaunay's time is:  0.042
----------
Fortune's algorith time is:  0.43
Delaunay's time is:  3.808
----------
Fortune's algorith time is:  0.531
Delaunay's time is:  15.004
----------
